In [ ]:
import os
#os.listdir('/home/em-student/experiment_boom/arch_cov_1/leaks/sim_input/')
os.listdir('/home/em-student/experiment_1/ct_cov_1/leaks/sim_input/')


In [ ]:
# input files
#in_file = '/home/em-student/experiment_1/ct_cov_1/leaks/sim_input/id_606718.si'
in_file = '/home/em-student/experiment_boom/arch_cov_1/leaks/sim_input/id_8536.si'


In [ ]:
# config
import tempfile
from src.utils import *
from Config import ROCKET_CONF, BOOM_CONF
outdir = tempfile.mkdtemp()


(mutator, preprocessor, hscHost) = setupHSC('Template', outdir, proc_num=0, debug=True, contract='ct', isa='RV64IM')

(sim_input, (data_a, data_b), assert_intr) = mutator.read_siminput(in_file)

(hsc_input, rtl_input, symbols) = preprocessor.process(sim_input, data_a, data_b, False)
stop = [ proc_state.NORMAL ]

In [ ]:
# process and run
args = [ hscHost.sail ] + hscHost.sail_args + ['-vplatform']#, '-vinstr']
args_a = args + [ hsc_input.binary_a, '-o', hscHost.out_a]
result = subprocess.run(args_a, stdout=subprocess.PIPE)

# Get the stdout and stderr from the result
stdout = result.stdout.decode()

In [ ]:
print(stdout)

In [ ]:
fb = open(os.path.join(outdir, '.input_0_a.S'))
fb_lines = fb.readlines()
for l in fb_lines:
    print(l)

In [ ]:
hscHost.sail_args

In [ ]:
os.listdir(outdir)

In [ ]:
import re

def extract_X(string):
    pattern = r'^handling exc#0x0((?!8|9|b|B)\w)'
    matches = re.findall(pattern, string, re.MULTILINE)
    return matches


In [ ]:
extract_X(stdout)

In [ ]:
list(map(lambda x: ord(x), ['7', '8', '9', 'A', 'b']))